In [1]:
import numpy as np

In [2]:
# ==========================================================
# LAMINA CLASS
# ==========================================================

class Lamina:
    def __init__(self, E1, E2, G12, nu12,
                 thickness,
                 theta_deg,
                 alpha1=0, alpha2=0,
                 beta1=0, beta2=0):

        self.E1 = E1
        self.E2 = E2
        self.G12 = G12
        self.nu12 = nu12
        self.nu21 = (E2/E1)*nu12
        self.t = thickness
        self.theta = np.radians(theta_deg)

        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.beta1 = beta1
        self.beta2 = beta2

    # Reduced stiffness matrix
    def Q(self):
        denom = 1 - self.nu12*self.nu21
        Q11 = self.E1 / denom
        Q22 = self.E2 / denom
        Q12 = self.nu12 * self.E2 / denom
        Q66 = self.G12

        return np.array([
            [Q11, Q12, 0],
            [Q12, Q22, 0],
            [0,   0,   Q66]
        ])

    # Transformed stiffness matrix
    def Qbar(self):
        m = np.cos(self.theta)
        n = np.sin(self.theta)
        Q = self.Q()

        T1 = np.array([
            [m**2,  n**2,  2*m*n],
            [n**2,  m**2, -2*m*n],
            [-m*n,   m*n,  m**2 - n**2]
        ])
        T2 = np.array([
            [m**2,   n**2,   m*n],
            [n**2,   m**2,  -m*n],
            [-2*m*n, 2*m*n,  m**2 - n**2]
        ])

        return np.linalg.inv(T1) @ Q @ T2  

    def transform_expansion(self, alpha1, alpha2):
        m = np.cos(self.theta)
        n = np.sin(self.theta)

        T_eps = np.array([
            [m**2,   n**2,   m*n],
            [n**2,   m**2,  -m*n],
            [-2*m*n, 2*m*n,  m**2 - n**2]
        ])

        return np.linalg.inv(T_eps) @ np.array([alpha1, alpha2, 0])  # Inverted



In [3]:
# ==========================================================
# LAMINATE CLASS
# ==========================================================

class Laminate:
    def __init__(self, plies):
        self.plies = plies
        self.n = len(plies)
        self.z = self._compute_z()

    def _compute_z(self):
        total_t = sum(p.t for p in self.plies)
        z = [-total_t/2]
        for p in self.plies:
            z.append(z[-1] + p.t)
        return z

    def ABD(self):
        A = np.zeros((3,3))
        B = np.zeros((3,3))
        D = np.zeros((3,3))

        for k in range(self.n):
            Qb = self.plies[k].Qbar()
            z_bot = self.z[k]
            z_top = self.z[k+1]

            A += Qb * (z_top - z_bot)
            B += 0.5 * Qb * (z_top**2 - z_bot**2)
            D += (1/3) * Qb * (z_top**3 - z_bot**3)

        return A, B, D

    def ABD_matrix(self):
        A, B, D = self.ABD()
        top = np.hstack((A, B))
        bottom = np.hstack((B, D))
        return np.vstack((top, bottom))

    # ------------------------------------------------------
    # Solve for midplane strain and curvature
    # ------------------------------------------------------

    def solve_for_strain(self, N, M):
        ABD = self.ABD_matrix()
        loads = np.concatenate((N, M))
        solution = np.linalg.inv(ABD) @ loads
        epsilon0 = solution[:3]
        kappa = solution[3:]
        return epsilon0, kappa

    # ------------------------------------------------------
    # Solve for loads given strain/curvature
    # ------------------------------------------------------

    def solve_for_strain(self, N, M, dT=0, dC=0):
        ABD = self.ABD_matrix()
        
        # Calculate thermal and moisture loads
        NT, MT = self.thermal_loads(dT)
        NM, MM = self.moisture_loads(dC)
        
        # Total environmental loads
        N_env = NT + NM
        M_env = MT + MM
        
        # Effective loads: applied loads minus environmental loads
        N_eff = N - N_env
        M_eff = M - M_env
        
        loads = np.concatenate((N_eff, M_eff))
        solution = np.linalg.inv(ABD) @ loads
        epsilon0 = solution[:3]
        kappa = solution[3:]
        return epsilon0, kappa

    # ------------------------------------------------------
    # Thermal residual loads
    # ------------------------------------------------------

    def thermal_loads(self, dT):
        NT = np.zeros(3)
        MT = np.zeros(3)

        for k in range(self.n):
            ply = self.plies[k]
            Qb = ply.Qbar()

            alpha = ply.transform_expansion(ply.alpha1, ply.alpha2)

            z_bot = self.z[k]
            z_top = self.z[k+1]

            NT += Qb @ alpha * dT * (z_top - z_bot)
            MT += Qb @ alpha * dT * 0.5 * (z_top**2 - z_bot**2)

        return NT, MT

    # ------------------------------------------------------
    # Hygroscopic residual loads
    # ------------------------------------------------------

    def moisture_loads(self, dC):
        NM = np.zeros(3)
        MM = np.zeros(3)

        for k in range(self.n):
            ply = self.plies[k]
            Qb = ply.Qbar()

            beta = ply.transform_expansion(ply.beta1, ply.beta2)

            z_bot = self.z[k]
            z_top = self.z[k+1]

            NM += Qb @ beta * dC * (z_top - z_bot)
            MM += Qb @ beta * dC * 0.5 * (z_top**2 - z_bot**2)

        return NM, MM

    # ------------------------------------------------------
    # Stress through thickness
    # ------------------------------------------------------
    def stresses(self, epsilon0, kappa, dT=0, dC=0):
        stress_data = []

        for k in range(self.n):
            ply = self.plies[k]
            Qb = ply.Qbar()

            z_mid = 0.5 * (self.z[k] + self.z[k+1])
            strain = epsilon0 + z_mid * kappa

            alpha = ply.transform_expansion(ply.alpha1, ply.alpha2)
            beta = ply.transform_expansion(ply.beta1, ply.beta2)

            strain_total = strain - alpha*dT - beta*dC
            stress = Qb @ strain_total

            # OPTIONAL: Add principal stress and failure index
            sigma11, sigma22, sigma12 = stress
            
            stress_data.append({
                'ply': k+1, 
                'z_mid': z_mid, 
                'stress': stress,
                'sigma11': sigma11,
                'sigma22': sigma22,
                'sigma12': sigma12
            })

        return stress_data


In [4]:
# ==========================================================
# LAMINATE BUILDERS
# ==========================================================

def build_laminate(sequence, material_props):
    plies = []
    for theta in sequence:
        plies.append(
            Lamina(
                material_props[0],  # E1
                material_props[1],  # E2
                material_props[2],  # G12
                material_props[3],  # nu12
                material_props[4],  # thickness
                theta,              # theta_deg
                material_props[5],  # alpha1
                material_props[6],  # alpha2
                material_props[7],  # beta1
                material_props[8]   # beta2
            )
        )
    return Laminate(plies)

In [5]:
# ==========================================================
# EXAMPLE MATERIAL (Carbon/Epoxy Typical)
# ==========================================================

material = (
    135e9,     # E1
    10e9,      # E2
    5e9,       # G12
    0.3,       # nu12
    0.125e-3,  # thickness
    -0.5e-6,   # alpha1
    25e-6,     # alpha2
    0.02e-3,   # beta1
    0.3e-3     # beta2
)


In [6]:
# ==========================================================
# REQUIRED CONFIGURATIONS
# ==========================================================

def run_analysis():

    laminates = {
        "[0]n (n=4)": [0,0,0,0],
        "[90]n (n=4)": [90,90,90,90],
        "[0/90]T": [0,90],
        "[0/90]s": [0,90,90,0],
        "[+45/-45]": [45,-45],
        "[+45/-45/-45/+45]": [45,-45,-45,45],
        "Balanced Quasi-Isotropic":
            [0,45,-45,90,90,-45,45,0]
    }

    for name, seq in laminates.items():
        print("\n====================================")
        print("Laminate:", name)
        print("Stacking:", seq)

        lam = build_laminate(seq, material)

        A, B, D = lam.ABD()

        print("\nA Matrix:\n", A)
        print("\nB Matrix:\n", B)
        print("\nD Matrix:\n", D)

        # Example mechanical load
        N = np.array([1e5, 0, 0])
        M = np.array([0,0,0])

        eps0, kappa = lam.solve_for_strain(N, M)

        print("\nMidplane Strain:", eps0)
        print("Curvature:", kappa)

        # Thermal residual
        NT, MT = lam.thermal_loads(-100)
        print("\nThermal Resultants NT:", NT)
        print("Thermal Moments MT:", MT)

        # Moisture residual
        NM, MM = lam.moisture_loads(0.02)
        print("\nMoisture Resultants NM:", NM)
        print("Moisture Moments MM:", MM)


if __name__ == "__main__":
    run_analysis()


Laminate: [0]n (n=4)
Stacking: [0, 0, 0, 0]

A Matrix:
 [[67953020.13422818  1510067.11409396        0.        ]
 [ 1510067.11409396  5033557.04697987        0.        ]
 [       0.                0.          2500000.        ]]

B Matrix:
 [[0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 2.84217094e-14 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00]]

D Matrix:
 [[1.41568792 0.03145973 0.        ]
 [0.03145973 0.10486577 0.        ]
 [0.         0.         0.05208333]]

Midplane Strain: [ 0.00148148 -0.00044444  0.        ]
Curvature: [-2.69479912e-18  1.21265960e-16  0.00000000e+00]

Thermal Resultants NT: [  -377.51677852 -12508.38926174      0.        ]
Thermal Moments MT: [0. 0. 0.]

Moisture Resultants NM: [36.24161074 30.80536913  0.        ]
Moisture Moments MM: [0. 0. 0.]

Laminate: [90]n (n=4)
Stacking: [90, 90, 90, 90]

A Matrix:
 [[5.03355705e+06 1.51006711e+06 9.04101664e-11]
 [1.51006711e+06 6.79530201e+07 3.76229579e-09]
 [9.04101664e-11 3